In [48]:
import time
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob
from scipy import stats

In [49]:
def VideoPull(back_frame, current_frame, file_name, path):
    video = [(os.path.join(root, file), file[0:-4]) 
        for root, dirs, files in os.walk(path) 
        for file in files 
        if file.endswith(file_name + '.mp4')]
    cam = cv2.VideoCapture(video[0][0])
    cam.set(1,back_frame)
    ret, f = cam.read(1)
    background = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    cam.set(1,current_frame)
    ret, c = cam.read(1)
    current = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
    difference = cv2.subtract(current,background)

    return background, current, difference, c

In [50]:
def Frame(name, df):
    start, end = df[df.name.str.contains(name)].start_frame.values, df[df.name.str.contains(name)].end_frame.values
    end = [int(elem) for elem in end]
    return(start,end)

In [51]:
video_list = r"E:\Downloads\ManducaMultiSenseData\Videos"
window_path = r"E:\Downloads\ManducaMultiSenseData\Motion\Centroids_Filtered\window\filteredFrameWindow.csv"
centroids = r"E:\Downloads\ManducaMultiSenseData\Motion\Centroids_Filtered"

In [52]:
names = ['L0.1_c-3_m32_16_149643', # moth sits on flower
 'L0.1_c-3_m32_17_154000', # moth sits on flower
 'L0.1_c-3_m34_2_29617', # moth sits on flower
 'L0.1_c-3_m34_11_45114', # moth sits on flower
 'L0.1_c-3_m34_14_51383', # moth sits on flower
 'L0.1_c-3_m34_22_74562', # already filtered out
 'L0.1_c-3_m34_25_81816', # already filtered out
#  'L0.1_c-3_m38_4_42921',  removed since moth and tracking were working well
#  'L0.1_c-3_m40_9_31581',  removed since moth and tracking were working well
#  'L0.1_c-3_m40_15_46420', removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames
#  'L0.1_c-3_m40_17_65761', removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames
#  'L0.1_c-3_m41_5_58180',  removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames
#  'L0.1_c-3_m41_6_61746',  removed since moth and tracking were working well
#  'L0.1_c-3_m41_7_66022',  removed since moth and tracking were working well
#  'L0.1_c-3_m41_8_69353',  removed since moth and tracking were working well
#  'L0.1_c-3_m41_9_71991',  removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames
#  'L0.1_c-3_m41_11_76152', removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames
#  'L0.1_c-3_m44_23_103079',removed since moth and tracking were working well
 'L0.1_c-3_m45_5_40142', # moth sits on flower
 'L0.1_c-3_m46_13_47364',# moth sits on flower
 'L0.1_c-3_m47_4_22585', # abnormal moth behavior
#  'L0.1_c-3_m47_30_74413', removed since moth and tracking were working well
 'L0.1_c-3_m5_0_26767',  # moth sits on flower
 'L0.1_c-3_m5_3_36270',  # moth sits on flower
 'L0.1_c-3_m5_4_41258',  # moth sits on flower
 'L0.1_c-3_m5_7_47239',  # moth sits on flower
 'L0.1_c-3_m5_8_48961',  # moth sits on flower
 'L0.1_c-3_m5_10_51462', # moth sits on flower
 'L0.1_c-3_m5_12_54487', # moth sits on flower
 'L0.1_c-3_m5_15_58748', # moth sits on flower
 'L0.1_c-3_m5_17_60830', # moth sits on flower
 'L0.1_c-3_m5_18_62363', # moth sits on flower
 'L0.1_c-3_m5_20_64412', # moth sits on flower
#  'L0.1_c-3_m5_21_65154',  removed since moth and tracking were working well
 'L0.1_c-3_m5_22_67013', # moth sits on flower
 'L0.1_c-3_m5_23_68970', # moth sits on flower
#  'L0.1_c-3_m5_24_69548',  removed since moth and tracking were working well. A leg or two touches the flower but the moth appears to hover the entire time
 'L0.1_c-3_m5_25_70531', # moth sits on flower
 'L0.1_c-3_m5_26_73715', # moth sits on flower
 'L0.1_c-3_m5_28_77295', # edge case moth very briefly sits on flower 2-3 times not going to include to play it safe 
 'L0.1_c-3_m5_29_79654', # moth sits on flower
 'L0.1_c-3_m5_30_81737', # moth sits on flower
 'L0.1_c-3_m5_31_83229', # moth sits on flower
 'L0.1_c-3_m5_32_84803', # moth sits on flower
 'L0.1_c-3_m5_36_98169', # moth sits on flower
 'L0.1_c-3_m5_37_99770', # moth sits on flower
#  'L0.1_c-3_m57_30_125307', removed since moth and tracking were working well. A leg or two touches the flower but the moth appears to hover the entrie time. Last frame
#  'L0.1_c-3_m57_31_130992', removed since moth and tracking were working well. A leg or two touches the flower but the moth appears to hover the entrie time. Last frame
#  'L0.1_c-3_m57_34_138460', removed since moth and tracking were working well
#  'L0.1_c-3_m8_4_35757', removed since moth and tracking were working well
 'L50_c-3_m22_19_82619', #flower getting picked up
 'L50_c-3_m35_7_49157' , # flower getting picked up
 'L50_c-3_m51_8_39608' ] # abnormal moth behavior

In [67]:
csv_list = glob.glob(centroids + '\\*.csv')
csv_list = [elem for elem in csv_list if elem[61:-4] in names]
len(csv_list), len(names)

(2, 2)

In [68]:
for name in csv_list:
    
    df = pd.read_csv(name)
    
    name = name[61:-4]
    
    window = pd.read_csv(window_path)
    
    if np.any(window.name.str.contains(str(name))):
    
        start,end = Frame(name,window)



        frameNum = np.arange(start,end)




        x,y = df.X_position.values, df.Y_position.values

        video_name = video_list +  "//PotentialOutlier/" + name  + '.mp4'

        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

        video = cv2.VideoWriter(video_name, fourcc, 30, (648,488))






        if name.startswith('L0.1'):
            name = name[:12]
            if name.endswith('_'):
                name = name[:-1]
        elif name.startswith('L50'):
            name = name[:11]
            if name.endswith('_'):
                name = name[:-1]


        print(name)



        i = 0


        while i < (len(x)-1):

    #         fig = plt.figure(figsize = (20,6))
            back, cur, diff, f = VideoPull(0,frameNum[i], name, video_list)
            c = [0,0,255]
            if np.isnan(x[i]):
                x[i], y[i] = 0,0
                c = [0,255,0]
            cv2.circle(f, (int(x[i]*74/2.5),int(y[i]*74/2.5)), 6, c, 3)

            i+=1
    #         plt.imshow(cur, cmap = 'gray')
            video.write(f)
    #         cv2.imshow('video',cur)
    #         if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
    #             break

        cv2.destroyAllWindows()
        video.release()
    else:
        print("already filtered out " + str(name))

already filtered outL0.1_c-3_m34_22_74562
already filtered outL0.1_c-3_m34_25_81816


In [53]:
csv_list

['E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m32_16_149643.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m32_17_154000.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m34_11_45114.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m34_14_51383.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m34_22_74562.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m34_25_81816.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m34_2_29617.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m38_4_42921.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m40_15_46420.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_Filtered\\L0.1_c-3_m40_17_65761.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\

In [54]:
remove_list = r'E:\Downloads\ManducaMultiSenseData\Videos\PotentialOutlier\\'
names = glob.glob(remove_list + "*.mp4")

In [59]:
name_list = []
for n in names:
    name = "\\" + n[59:-4] + ".csv"
    name_list.append(name)

In [60]:
name_list

['\\L0.1_c-3_m23_6_66939.csv',
 '\\L0.1_c-3_m32_16_149643.csv',
 '\\L0.1_c-3_m32_17_154000.csv',
 '\\L0.1_c-3_m34_11_45114.csv',
 '\\L0.1_c-3_m34_14_51383.csv',
 '\\L0.1_c-3_m34_19_67675.csv',
 '\\L0.1_c-3_m34_2_29617.csv',
 '\\L0.1_c-3_m37_0_36557.csv',
 '\\L0.1_c-3_m38_0_24374.csv',
 '\\L0.1_c-3_m39_10_40618.csv',
 '\\L0.1_c-3_m39_11_41749.csv',
 '\\L0.1_c-3_m39_12_43448.csv',
 '\\L0.1_c-3_m39_13_44002.csv',
 '\\L0.1_c-3_m39_2_31673.csv',
 '\\L0.1_c-3_m39_3_32250.csv',
 '\\L0.1_c-3_m39_4_35326.csv',
 '\\L0.1_c-3_m39_5_35876.csv',
 '\\L0.1_c-3_m39_6_37544.csv',
 '\\L0.1_c-3_m39_7_37856.csv',
 '\\L0.1_c-3_m39_8_39080.csv',
 '\\L0.1_c-3_m39_9_40013.csv',
 '\\L0.1_c-3_m45_5_40142.csv',
 '\\L0.1_c-3_m46_13_47364.csv',
 '\\L0.1_c-3_m47_4_22585.csv',
 '\\L0.1_c-3_m5_0_26767.csv',
 '\\L0.1_c-3_m5_10_51462.csv',
 '\\L0.1_c-3_m5_12_54487.csv',
 '\\L0.1_c-3_m5_15_58748.csv',
 '\\L0.1_c-3_m5_17_60830.csv',
 '\\L0.1_c-3_m5_18_62363.csv',
 '\\L0.1_c-3_m5_20_64412.csv',
 '\\L0.1_c-3_m5_22_67013.csv